# Project 2

## Predicting NFL All-Pros

In [20]:
import pandas as pd

In [21]:
url = 'https://www.pro-football-reference.com/years/2021/allpro.htm'

pd.read_html(url)[0]

,Pos,Player,Tm,Age,Yrs,G,GS,Cmp,Att,Yds,...,Att.1,Yds.1,TD.1,Rec,Yds.2,TD.2,Solo,Sk,Int.1,All-pro teams
0,QB,Aaron Rodgers,GNB,38,16,16,16,366,531,4115,...,33,101,3,1,-4,0,0,0.0,0,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, ..."
1,QB,Tom Brady,TAM,44,21,17,17,485,719,5316,...,28,81,2,0,0,0,0,0.0,0,"AP: 2nd Tm, PFF: 1st Tm"
2,QB,Joe Burrow,CIN,25,1,16,16,366,520,4611,...,40,118,2,0,0,0,1,0.0,0,"FW: 1st Tm All-Conf., PFF: 2nd Tm"
3,RB,Jonathan Taylor,IND,22,1,17,17,0,0,0,...,332,1811,18,40,360,2,0,0.0,0,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, ..."
4,RB,Nick Chubb,CLE,26,3,14,14,0,0,0,...,228,1259,8,20,174,1,0,0.0,0,"FW: 1st Tm All-Conf., FW: 1st Tm"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,ST,Ashton Dulin,IND,24,2,17,1,0,0,0,...,3,32,0,13,173,2,12,0.0,0,"AP: 2nd Tm, PFF: 2nd Tm"
83,PK,Justin Tucker,BAL,32,9,17,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, ..."
84,PK,Matt Gay,LAR,27,2,17,0,0,0,0,...,0,0,0,0,0,0,1,0.0,0,FW: 1st Tm All-Conf.
85,FLEX-D,Jevon Holland,MIA,21,Rook,16,13,0,0,0,...,0,0,0,0,0,0,48,2.5,2,PFF: 2nd Tm


In [22]:
df_ap = pd.DataFrame()

for year in range(1970, 2022):
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/allpro.htm'

    df_url = pd.read_html(url)[0]

    df_url.drop(columns = ['Yrs', 'Cmp', 'Att', 'Yds', 'TD', 'Int', 'Att.1', 'Yds.1', 'TD.1', 
                           'Rec', 'Yds.2', 'TD.2', 'Solo', 'Sk', 'Int.1'], inplace = True)
    
    df_url['Year'] = year

    df_temp = [df_ap, df_url]

    df_ap = pd.concat(df_temp)
    


In [23]:
df_ap.rename(columns = {'Pos' : 'ap_pos', 'All-pro teams': 'all_pro'}, inplace = True)

In [24]:
df_ap_trimmed = df_ap.loc[df_ap['all_pro'].str.contains('AP')]

In [35]:
df_ap_trimmed['ap_1st'] = df_ap_trimmed.all_pro.apply(lambda teams : 1 if ' AP: 1st Tm' in teams.split(',') else 0)
df_ap_trimmed['ap_2nd'] = df_ap_trimmed.all_pro.apply(lambda teams : 1 if ' AP: 2nd Tm' in teams.split(',') else 0)

/tmp/ipykernel_1648/1741649708.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ap_trimmed['ap_1st'] = df_ap_trimmed.all_pro.apply(lambda teams : 1 if ' AP: 1st Tm' in teams.split(',') else 0)
/tmp/ipykernel_1648/1741649708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ap_trimmed['ap_2nd'] = df_ap_trimmed.all_pro.apply(lambda teams : 1 if ' AP: 2nd Tm' in teams.split(',') else 0)


In [37]:
df_ap = df_ap_trimmed[df_ap_trimmed['ap_1st'] + df_ap_trimmed['ap_2nd'] > 0]

,ap_pos,Player,Tm,Age,G,GS,all_pro,Year,ap_1st,ap_2nd
0,K,Jan Stenerud,KAN,28,14,0,"AP: 1st Tm All-Conf., SN: 1st Tm All-Conf., AP...",1970,1,0
1,LLB,Bobby Bell,KAN,30,14,14,"AP: 1st Tm All-Conf., PFW: 1st Tm All-Conf., S...",1970,1,0
2,QB,John Brodie,SFO,35,14,14,"AP: 1st Tm All-Conf., PFW: 1st Tm All-Conf., S...",1970,1,0
3,RB,Larry Brown,WAS,23,13,13,"AP: 1st Tm All-Conf., PFW: 1st Tm All-Conf., S...",1970,1,0
4,MLB,Dick Butkus,CHI,28,14,14,"AP: 1st Tm All-Conf., PFW: 1st Tm All-Conf., S...",1970,1,0
...,...,...,...,...,...,...,...,...,...,...
75,PR,Jakeem Grant,2TM,29,15,0,"FW: 1st Tm All-Conf., AP: 2nd Tm, SN: 1st Tm",2021,0,1
76,KR,Braxton Berrios,NYJ,26,16,1,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, ...",2021,1,0
79,ST,J.T. Gray,NOR,25,16,0,"FW: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm",2021,1,0
81,ST,Matthew Slater,NWE,36,17,0,"FW: 1st Tm All-Conf., AP: 2nd Tm",2021,0,1
